In [142]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns

# PyRQP

## Input parameters

In [143]:
# Flows
riv_flow_mean = 100
riv_flow_95pc = 20
dis_flow_mean = 20
dis_flow_sd = 8

# Water quality
riv_wq_mean = 2
riv_wq_sd = 1
dis_wq_mean = 15
dis_wq_sd = 7

# Correlations
corr_riv_dis_flow = 0.6
corr_riv_flow_wq = -0.3
corr_dis_flow_wq = -0.2

## Prepare all functionality

In [144]:
def transform_log_to_normal(lg_mean, lg_sd):
    """ 
    Transformation from log mean and sd to normal 
    mean and sd using the method of moments
    """
    mean = np.log(lg_mean / ((1 + ((lg_sd**2) / (lg_mean**2))) ** 0.5))
    sd = (np.log(1 + (lg_sd**2) / (lg_mean**2))) ** 0.5
    return mean, sd

In [145]:
def calculate_covariance(corr, std_1, std_2):
    """
    This formula takes a correlation and two std
    and calculates the covariance matrix
    """
    cov = corr * std_1 * std_2
    return cov

In [146]:
def calculate_multivariate_log_normal(
    mean1, std1, mean2, std2, mean3, std3, mean4, std4, corr1_2, corr1_3, corr2_4
):
    """ """
    # Transform to normal
    mean1, std1 = transform_log_to_normal(mean1, std1)
    mean2, std2 = transform_log_to_normal(mean2, std2)
    mean3, std3 = transform_log_to_normal(mean3, std3)
    mean4, std4 = transform_log_to_normal(mean4, std4)

    # Calculate covariances
    cov1_2 = calculate_covariance(corr1_2, std1, std2)
    cov1_3 = calculate_covariance(corr1_3, std1, std3)
    cov2_4 = calculate_covariance(corr2_4, std2, std4)

    # Build covariance matrix
    cov_matrix = [
        [std1**2, cov1_2, cov1_3, 0],
        [cov1_2, std2**2, 0, cov2_4],
        [cov1_3, 0, std3**2, 0],
        [0, cov2_4, 0, std4**2],
    ]
    cov_matrix = np.array(cov_matrix)

    # Generate normal random multivariate
    data = np.random.multivariate_normal(
        [mean1, mean2, mean3, mean4], cov_matrix, size=100000
    )

    # Transform to lognormal
    data = np.exp(data)

    df = pd.DataFrame(data, columns=["riv_flow", "dis_flow", "riv_qual", "dis_qual"])

    return df, cov_matrix

In [147]:
def calculate_log_mean_sd_from_95pc(lg_mean, lg_95pc):
    """
    Calculate the underlying normal sd from the lognormal
    mean and 95th low flow percentile
    """
    sd = (2.705543 + 2*np.log(lg_mean) - 2*np.log(lg_95pc)) ** 0.5 - 1.644854
    mean = np.log(lg_mean) - 0.5 * (sd**0.5)
    lg_sd = lg_mean * (np.exp(sd**2) - 1) ** 0.5
    return lg_mean, lg_sd

#### Demonstration

Let's call the (as yet unknown) mean and standard deviation of the underlying normal $\mu$ and $\sigma$, so $X \sim N(\mu,\sigma^2)$, and the (known) mean of the lognormal $m_Y$ with $5th$ percentile value $Y_{(0.05)}$, where $Y=e^X$.

Then we should have:

(1) $$m_Y=\exp(\mu+\frac12\sigma^2)$$

(2) $$Y_{(0.05)} = \exp(\mu+ \Phi^{-1}(0.05) \sigma)$$

where $\Phi^{-1}(0.05)$ is the $5th$ percentile value of a standard normal, about $−1.644854$

Taking the log:

(1) $$\log(m_Y)=\mu+\frac12\sigma^2$$

(2) $$\log(Y_{(0.05)})=\mu+ \Phi^{-1}(0.05) \sigma$$

We solve (1) for $\mu$ and substitute on (2), then solve (2) for $\sigma$ by using the quadratic formula

## Calculate downstream quality

In [148]:
_, riv_flow_sd = calculate_log_mean_sd_from_95pc(riv_flow_mean, riv_flow_95pc)

In [149]:
df, cov_matrix = calculate_multivariate_log_normal(
    # Flow
    riv_flow_mean,
    riv_flow_sd,
    dis_flow_mean,
    dis_flow_sd,
    # Quality
    riv_wq_mean,
    riv_wq_sd,
    dis_wq_mean,
    dis_wq_sd,
    # Correlations
    corr_riv_dis_flow,
    corr_riv_flow_wq,
    corr_dis_flow_wq,
)

In [150]:
df = df.eval("ds_flow = riv_flow + dis_flow")
df = df.eval("ds_qual = (riv_flow * riv_qual + dis_flow * dis_qual) / ds_flow")

## Calculate descriptive statistics

In [151]:
stats = df.agg(["mean", "std"]).T
stats["90pc"] = df.quantile(0.90)
stats["95pc"] = df.quantile(0.95)
stats["99pc"] = df.quantile(0.99)
stats

,mean,std,90pc,95pc,99pc
riv_flow,99.407866,91.287319,201.608063,267.894528,449.804612
dis_flow,19.989122,8.050631,30.455272,35.092568,45.751243
riv_qual,2.000222,0.999722,3.276808,3.881448,5.364351
dis_qual,14.978053,6.958193,24.009860,28.054329,37.680879
ds_flow,119.396988,95.869388,228.094762,296.504745,484.435910
ds_qual,4.748234,2.276109,7.649315,9.013778,12.411726


In [152]:
df.corr()  # This is important information that should be part of the analysis

,riv_flow,dis_flow,riv_qual,dis_qual,ds_flow,ds_qual
riv_flow,1.000000,0.539346,-0.227150,0.000881,0.997497,-0.462078
dis_flow,0.539346,1.000000,0.005186,-0.179433,0.597543,-0.188854
riv_qual,-0.227150,0.005186,1.000000,-0.004451,-0.215858,0.527656
dis_qual,0.000881,-0.179433,-0.004451,1.000000,-0.014229,0.579439
ds_flow,0.997497,0.597543,-0.215858,-0.014229,1.000000,-0.455852
ds_qual,-0.462078,-0.188854,0.527656,0.579439,-0.455852,1.000000


## Backward calculation

In [153]:
target = 6
percentile = 0.9
# TODO Look at equation from notebook to transform from percentile to mean

In [154]:
# Calculate scale factor and scale
scale = target / df["ds_qual"].quantile(percentile)
print(scale)
df["ds_qual_target"] = df["ds_qual"] * scale
# Recalculate discharge quality target
df = df.eval("dis_qual_target = (ds_flow * ds_qual_target - riv_flow * riv_qual) / dis_flow")
# Recalculate dis_qual_target based keeping CoV
adj_factor = df["dis_qual_target"].mean() / df["dis_qual"].mean()
df["dis_qual_target"] = df["dis_qual"] * adj_factor
# Re-calculate ds water quality
df = df.eval("ds_qual_target = (riv_flow * riv_qual + dis_flow * dis_qual_target) / ds_flow")
# Check the scale
scale = target / df["ds_qual_target"].quantile(percentile)

0.784383991587308


In [155]:
while round(scale, 4) != 1:
    print(scale)
    # Calculate scale factor and scale
    df["ds_qual_target"] = df["ds_qual_target"] * scale
    # Recalculate discharge quality target
    df = df.eval("dis_qual_target = (ds_flow * ds_qual_target - riv_flow * riv_qual) / dis_flow")
    # Recalculate dis_qual_target based keeping CoV
    adj_factor = df["dis_qual_target"].mean() / df["dis_qual"].mean()
    df["dis_qual_target"] = df["dis_qual"] * adj_factor
    # Re-calculate ds water quality
    df = df.eval("ds_qual_target = (riv_flow * riv_qual + dis_flow * dis_qual_target) / ds_flow")
    # Check the scale
    scale = target / df["ds_qual_target"].quantile(percentile)

1.0494775858087015
0.9897962498643121
1.0022521135576778
0.9994392653405187
1.0001234181359155


In [156]:
stats = df.agg(["mean", "std"]).T
stats["90pc"] = df.quantile(0.90)
stats["95pc"] = df.quantile(0.95)
stats["99pc"] = df.quantile(0.99)
stats["99.5pc"] = df.quantile(0.995)
stats["cov"] = stats["std"] / stats["mean"]
stats

,mean,std,90pc,95pc,99pc,99.5pc,cov
riv_flow,99.407866,91.287319,201.608063,267.894528,449.804612,543.181136,0.918311
dis_flow,19.989122,8.050631,30.455272,35.092568,45.751243,50.563864,0.402751
riv_qual,2.000222,0.999722,3.276808,3.881448,5.364351,6.016026,0.499806
dis_qual,14.978053,6.958193,24.009860,28.054329,37.680879,42.031679,0.464559
ds_flow,119.396988,95.869388,228.094762,296.504745,484.435910,581.879578,0.802946
ds_qual,4.748234,2.276109,7.649315,9.013778,12.411726,13.964811,0.479359
ds_qual_target,3.814799,1.703766,5.999950,7.010747,9.421602,10.462469,0.446620
dis_qual_target,10.643650,4.944606,17.061800,19.935866,26.776651,29.868401,0.464559


In [157]:
# https://stats.stackexchange.com/questions/212690/the-product-of-two-lognormal-random-variables
# https://stats.stackexchange.com/questions/627427/given-two-rvs-x-and-y-if-x-y-z-is-it-possible-to-change-the-mean-and-s?noredirect=1#comment1170511_627427
# https://stats.stackexchange.com/questions/381988/scaling-percentiles-of-log-normal-distribution
# https://stats.stackexchange.com/questions/344825/how-do-i-find-new-standard-deviation-from-two-means-and-their-sd
# https://stats.libretexts.org/Bookshelves/Probability_Theory/Probability_Mathematical_Statistics_and_Stochastic_Processes_(Siegrist)/05%253A_Special_Distributions/5.12%253A_The_Lognormal_Distribution